<a href="https://colab.research.google.com/github/sota1111/DL_Zero4/blob/main/DL_Zero4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os, sys
os.chdir('/content/drive/MyDrive/DL_Zero')

Mounted at /content/drive


In [3]:
!git config --global user.email "sota.moro@gmail.com"
!git config --global user.name "sota1111"

In [1]:
!pip install numpy matplotlib ipython scikit-learn pandas pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

# 1章 バンディット問題

## 1.1 機械学習の分類と強化学習

### 1.1.1 教師あり学習

### 1.1.2 教師なし学習

### 1.1.3 強化学習

## 1.2 バンディット問題

### 1.2.1 バンディット問題とは
バンディット：スロットマシンの別称  
バンディット問題における強化学習の枠組みを考える  
- 環境：スロットマシン
- エージェント：プレイヤー
- 行動：プレイヤーは複数台あるスロットマシンから1台を選んでプレイする。  
- 報酬：行動の結果、スロットマシンから受け取るコイン

### 1.2.2 良いスロットマシンとは
スロットマシンをプレイするような確率的事象は「期待値」を使って評価できる。

### 1.2.3 数式を使って表す
確率変数：取る値が確率的に決まる変数
- A：エージェントの行う行動
- E：期待値
- R：報酬
- E[R]：報酬Rの期待値
- E[R|A]：Aという行動を選んだ場合の報酬の期待値  
| の右側に条件を記述
- Q:行動価値
q(A)=E[R|A]

## 1.3 バンディットアルゴリズム
プレイヤーは報酬の期待値が分からない状態で、できるだけ精度良く推定することが求められる。

## 1.4 価値の推定方法
スロットマシンを実際にプレイして得られた報酬→標本平均

### 1.3.2 平均値を求める実装

In [6]:
import numpy as np

np.random.seed(0)
rewards = []

for n in

SyntaxError: ignored